# SageMaker Factorization Machine(FM)으로 추천 시스템 구축하기

*본 노트북 예제는 AWS 머신 러닝 블로그에 기고된 글들에 기반하여 SageMaker의 Factorization Machine(FM)으로 추천 시스템을 구축하는 방법을 설명합니다.*

References
- [Build a movie recommender with factorization machines on Amazon SageMaker](https://aws.amazon.com/ko/blogs/machine-learning/build-a-movie-recommender-with-factorization-machines-on-amazon-sagemaker/)
- [Amazon SageMaker Factorization Machines 알고리즘을 확장하여 추천 시스템 구현하기](https://aws.amazon.com/ko/blogs/korea/extending-amazon-sagemaker-factorization-machines-algorithm-to-predict-top-x-recommendations/)
- [Factorization Machine 논문](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)

## 1. Factorization Machine
---

### 개요

일반적인 추천 문제들은 user가 행, item이 열, rating이 값으로 이루어진 행렬을 데이터셋으로 하여 Matrix Factorization 기법을 활용하는데, real-world의 다양한 메타데이터 피처(feature)들을 그대로 적용하기에는 어려움이 있습니다. Factoriztion Machine(이하 FM) 알고리즘은 Matrix Factorization의 개념을 확장하여 메타데이터 피처들을 같이 고려하고 피처 간의 상호 관계(interaction)를 선형 계산 복잡도로 자동으로 모델링할 수 있기에, 피처 엔지니어링에 들어가는 노력을 크게 줄일 수 있습니다.

### 설명

다양한 메타데이터 피처를 고려하기 위해 아래 그림처럼 user와 item을 원-핫 인코딩으로 변환하고 추가 피처들을 그대로 concatenate하여 `f(user, item, additional features) = rating` 형태의 선형 회귀(Linear Regression) 문제로 변환하여 풀 수 있습니다. 

![Factorization Machine](./fm.png "Factorization Machine")

하지만, 추천 문제를 선형 회귀로만 풀려고 하면 피처 간의 상호 관계를 고려할 수 없기에 아래 수식처럼 피처 간의 상호 관계를 모델링하는 항을 추가하여 다항 회귀(Polynomial Regression)로 변환해야 합니다.

$$
\begin{align} \hat{y}(\mathbf{x}) = w_{0} + \sum_{i=1}^{d} w_{i} x_{i} + \sum_{i=1}^d \sum_{j=i+1}^d x_{i} x_{j} w_{ij}, \;\; x \in \mathbb{R}^d \tag {1} 
\end{align}
$$ 
$d$는 피처 갯수로, $x \in \mathbb{R}^d$는 단일 샘플의 피처 벡터를 나타냅니다.

하지만 대부분의 추천 시스템 데이터셋은 희소하기에(sparse) cold-start 문제가 있으며, 추가적으로 고려해야 하는 피처들이 많아질 수록 계산이 매우 복잡해집니다. (예: user가 6만명, item 갯수가 5천개, 추가 피처가 5천개일 경우 70,000x70,000 행렬을 예측해야 합니다.)  

FM은 이러한 문제들을 행렬 분해 기법을 활용하여 feature 쌍(예: user, item) 간의 상호 관계를 내적(dot product)으로 변환하고 
수식을 재구성하여 계산 복잡도를 $O(kd^2)$에서 $O(kd)$로 감소시켰습니다. (수식 (2)에서 추가적인 계산을 거치면 계산 복잡도를 선형으로 감소할 수 있습니다. 자세한 내용은 논문을 참조하세요.) 

$$
\begin{align}
\hat{y}(\mathbf{x}) = w_{0} + \sum_{i=1}^{d} w_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d  x_{i} x_{j} \langle\mathbf{v}_i, \mathbf{v}_j\rangle \tag{2} 
\end{align}
$$

$$
\begin{align}
\langle \textbf{v}_i , \textbf{v}_{j} \rangle = \sum_{f=1}^k v_{i,f} v_{j,f},\; k: \text{dimension of latent feature}  \tag{3}
\end{align}
$$

위의 모델을 2-way(degree = 2) FM이라고 하며, 이를 일반화한 d-way FM도 있지만, 보통 2-way FM를 많이 사용합니다. SageMaker의 FM 또한 2-way FM입니다.

FM이 훈련하는 파라메터 튜플은 ($w_{0}, \mathbf{w}, \mathbf{V}$) 이며, 의미는 아래와 같습니다.
- $w_{0} \in \mathbb{R}$: global bias
- $\mathbf{w} \in \mathbb{R}^d$: 피처 벡터 $\mathbf{x}_i$의 가중치
- $\mathbf{V} \in \mathbb{R}^{n \times k}$: 피처 임베딩 행렬로 i번째 행은 $\mathbf{v}_i$


FM은 위의 수식에서 알 수 있듯이 closed form이며 시간 복잡도가 선형이기 때문에, 다수의 user & item과 메타데이터들이 많은 추천 문제에 적합합니다.
훈련 방법은 대표적으로 Gradient Descent, ALS(Alternating Least Square), MCMC(Markov Chain Monte Carlo)가 있으며, AWS에서는 이 중 딥러닝 아키텍처에 기반한 Gradient Descent를 MXNet 프레임워크를 이용하여 훈련합니다.

## 2. MovieLens 데이터셋으로 FM 모델 훈련 및 배포하기
---

딥러닝의 Hello World가 MNIST 데이터셋이라면, 추천 시스템의 Hello World는 MovieLens 데이터셋입니다.
이 데이터셋은 여러 크기로 제공되며, 본 예제에서는 943명의 사용자와 1,682개의 영화에 대해 10만개의 등급이 부여된 ml100k를 사용합니다.

In [1]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri
import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
import boto3, io, os, csv, json

### MovieLens 데이터셋 다운로드

In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2020-03-12 23:40:30--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  14.6MB/s    in 0.3s    

2020-03-12 23:40:31 (14.6 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

### 데이터 셔플링

In [3]:
!shuf ml-100k/ua.base -o ml-100k/ua.base.shuffled

### 훈련 데이터 로드

In [4]:
user_movie_ratings_train = pd.read_csv('ml-100k/ua.base.shuffled', sep='\t', index_col=False, 
                 names=['user_id' , 'movie_id' , 'rating'])
user_movie_ratings_train.head(5)

,user_id,movie_id,rating
0,416,762,3
1,574,332,3
2,854,483,4
3,280,1217,5
4,476,210,4


### 테스트 데이터 로드

In [5]:
user_movie_ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', index_col=False, 
                 names=['user_id' , 'movie_id' , 'rating'])
user_movie_ratings_test.head(5)

,user_id,movie_id,rating
0,1,20,4
1,1,33,4
2,1,61,4
3,1,117,3
4,1,155,2


10만건의 등급 데이터가 왜 희소한지 궁금하실 수 있습니다. 943명의 사용자와 1682개의 영화를 모두 고려하면 가능한 rating의 총 개수는
943 * 1,682 = 1,586,126개로 이 중 6.3%의 등급만 보유하게 됩니다. 

In [6]:
nb_users = user_movie_ratings_train['user_id'].max()
nb_movies = user_movie_ratings_train['movie_id'].max() 
nb_features = nb_users + nb_movies
total_ratings = nb_users * nb_movies

nb_ratings_test = len(user_movie_ratings_test.index)
nb_ratings_train = len(user_movie_ratings_train.index)

print("# of users: {}".format(nb_users))
print("# of movies: {}".format(nb_movies))
print("Training Count: {}".format(nb_ratings_train))
print("Test Count: {}".format(nb_ratings_test))
print("Features (# of users + # of movies): {}".format(nb_features))
print("Sparsity: {}%".format(((nb_ratings_test+nb_ratings_train)/total_ratings)*100))     

# of users: 943
# of movies: 1682
Training Count: 90570
Test Count: 9430
Features (# of users + # of movies): 2625
Sparsity: 6.304669364224531%


### 원-핫 인코딩 희소 행렬 변환

FM의 입력 데이터 포맷인 원-핫 인코딩 희소 행렬로 변환하겠습니다. 물론 희소 행렬이 아닌 밀집(dense) 행렬도 가능하지만, 데이터가 많아질수록 계산 속도가 느려지므로, 희소 행렬을 추천합니다. 

참고로, MovieLens 데이터셋은 별도의 메타데이터 피처가 존재하지 않아 943명의 사용자와 1,682개 영화에 대해서만 원-핫 인코딩 변환을 수행하므로 변환 후 피처의 차원은 943+1,682=2,625입니다.

또한, 본 예시에서는 rating이 4 이상인 영화들에 대한 이진 분류 문제로 간소화합니다. (즉, rating이 4 이상일 경우 $y = 1$, 4 미만일 경우 $y = 0$ 입니다.)

아래 셀은 약 20초 소요되며, 변환 후 데이터셋의 차원은 rating 개수 x 피쳐 개수 입니다.

In [7]:
%%time
def loadDataset(df, lines, columns):
    
    # 피처는 원-핫 인코딩 희소 행렬로 변환합니다.
    X = lil_matrix((lines, columns)).astype('float32')
    Y = []
    line = 0
    for line, (index, row) in enumerate(df.iterrows()):
            X[line,row['user_id']-1] = 1
            X[line, nb_users+(row['movie_id']-1)] = 1
            
            if int(row['rating']) >= 4:
                Y.append(1)
            else:
                Y.append(0)

    Y = np.array(Y).astype('float32')            
    return X,Y

X_train, Y_train = loadDataset(user_movie_ratings_train, nb_ratings_train, nb_features)
X_test, Y_test = loadDataset(user_movie_ratings_test, nb_ratings_test, nb_features)

CPU times: user 17.6 s, sys: 33.4 ms, total: 17.6 s
Wall time: 17.6 s


In [8]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nb_ratings_train, nb_features)
assert Y_train.shape == (nb_ratings_train, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: {} zeros, {} ones".format(zero_labels, nb_ratings_train-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nb_ratings_test, nb_features)
assert Y_test.shape  == (nb_ratings_test, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: {} zeros, {} ones".format(zero_labels, nb_ratings_test-zero_labels))

(90570, 2625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 2625)
(9430,)
Test labels: 5469 zeros, 3961 ones


### Protobuf 포맷 변환 후 S3에 저장

In [9]:
import sagemaker
bucket = sagemaker.Session().default_bucket()
#bucket = '[YOUR-BUCKET]'
prefix = 'fm-hol'

if bucket.strip() == '':
    raise RuntimeError("bucket name is empty.")

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

아래 셀은 약 15초 소요됩니다.

In [10]:
%%time
def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
fm_test_data_path  = writeDatasetToProtobuf(X_test, bucket, test_prefix, test_key, "sparse", Y_test)    
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)
print("FM model output S3 path: ", output_prefix)

Training data S3 path:  s3://sagemaker-us-east-1-143656149352/fm-hol/train/train.protobuf
Test data S3 path:  s3://sagemaker-us-east-1-143656149352/fm-hol/test/test.protobuf
FM model output S3 path:  s3://sagemaker-us-east-1-143656149352/fm-hol/output
CPU times: user 14.6 s, sys: 202 ms, total: 14.8 s
Wall time: 14.9 s


### 훈련

본 핸즈온은 하이퍼파라메터 튜닝 없이 휴리스틱한 하이퍼파라메터들을 사용합니다. 

- `feature_dim`: 피처 개수로 본 핸즈온에서는 2,625으로 설정해야 합니다.
- `mini_batch_size`: 본 핸즈온에서는 1,000으로 설정합니다.
- `num_factors`: latent factor 차원으로 본 핸즈온에서는 64차원으로 설정합니다.
- `epochs`: 본 핸즈온에서는 100 에폭으로 설정합니다.

In [11]:
instance_type_training = 'ml.c4.xlarge'
fm = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "factorization-machines"),
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type=instance_type_training,
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nb_features,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

이제 훈련을 시작하기 위한 모든 준비가 완료되었으며, 여러분께서는 `fit()` 메소드만 호출하면 됩니다. <br>
훈련은 약 4분에서 5분이 소요되며(순수 훈련에 소요되는 시간은 훨씬 짧지만, 훈련 인스턴스를 프로비저닝하는 시간이 고정적으로 소요됩니다), 검증 데이터셋에 대한 accuracy는 약 70%에 F1 스코어는 약 0.73~0.74입니다. (아래 output 메세지 참조)

```
[03/12/2020 09:35:42 INFO 139967441712960] #test_score (algo-1) : ('binary_classification_accuracy', 0.6928950159066808)
[03/12/2020 09:35:42 INFO 139967441712960] #test_score (algo-1) : ('binary_classification_cross_entropy', 0.5799107152103493)
[03/12/2020 09:35:42 INFO 139967441712960] #test_score (algo-1) : ('binary_f_1.000', 0.7331859222406486)
```

In [12]:
%%time
fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path})

2020-03-12 23:41:11 Starting - Starting the training job...
2020-03-12 23:41:12 Starting - Launching requested ML instances......
2020-03-12 23:42:17 Starting - Preparing the instances for training......
2020-03-12 23:43:18 Downloading - Downloading input data...
2020-03-12 23:43:57 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[03/12/2020 23:44:00 INFO 140601087346496] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_

[2020-03-12 23:44:08.658] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 24, "duration": 595, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:08 INFO 140601087346496] #quality_metric: host=algo-1, epoch=11, train binary_classification_accuracy <score>=0.715065934066
[03/12/2020 23:44:08 INFO 140601087346496] #quality_metric: host=algo-1, epoch=11, train binary_classification_cross_entropy <loss>=0.604453672304
[03/12/2020 23:44:08 INFO 140601087346496] #quality_metric: host=algo-1, epoch=11, train binary_f_1.000 <score>=0.76614205186
#metrics {"Metrics": {"update.time": {"count": 1, "max": 602.9999256134033, "sum": 602.9999256134033, "min": 602.9999256134033}}, "EndTime": 1584056648.658487, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056648.054763}

[03/12/2020 23:44:08 INFO 140601087346496] #progress_metric: host=algo-1, completed 12 % of epochs
#metrics {"Metrics":

[2020-03-12 23:44:18.623] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 58, "duration": 571, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:18 INFO 140601087346496] #quality_metric: host=algo-1, epoch=28, train binary_classification_accuracy <score>=0.732692307692
[03/12/2020 23:44:18 INFO 140601087346496] #quality_metric: host=algo-1, epoch=28, train binary_classification_cross_entropy <loss>=0.559710399586
[03/12/2020 23:44:18 INFO 140601087346496] #quality_metric: host=algo-1, epoch=28, train binary_f_1.000 <score>=0.769573248709
#metrics {"Metrics": {"update.time": {"count": 1, "max": 573.6351013183594, "sum": 573.6351013183594, "min": 573.6351013183594}}, "EndTime": 1584056658.624247, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056658.049838}

[03/12/2020 23:44:18 INFO 140601087346496] #progress_metric: host=algo-1, completed 29 % of epochs
#metrics {"Metrics"

[2020-03-12 23:44:28.721] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 92, "duration": 568, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:28 INFO 140601087346496] #quality_metric: host=algo-1, epoch=45, train binary_classification_accuracy <score>=0.738417582418
[03/12/2020 23:44:28 INFO 140601087346496] #quality_metric: host=algo-1, epoch=45, train binary_classification_cross_entropy <loss>=0.540485577594
[03/12/2020 23:44:28 INFO 140601087346496] #quality_metric: host=algo-1, epoch=45, train binary_f_1.000 <score>=0.771817484663
#metrics {"Metrics": {"update.time": {"count": 1, "max": 570.4739093780518, "sum": 570.4739093780518, "min": 570.4739093780518}}, "EndTime": 1584056668.72207, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056668.15082}

[03/12/2020 23:44:28 INFO 140601087346496] #progress_metric: host=algo-1, completed 46 % of epochs
#metrics {"Metrics": 

[2020-03-12 23:44:38.833] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 126, "duration": 579, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:38 INFO 140601087346496] #quality_metric: host=algo-1, epoch=62, train binary_classification_accuracy <score>=0.745703296703
[03/12/2020 23:44:38 INFO 140601087346496] #quality_metric: host=algo-1, epoch=62, train binary_classification_cross_entropy <loss>=0.529055700617
[03/12/2020 23:44:38 INFO 140601087346496] #quality_metric: host=algo-1, epoch=62, train binary_f_1.000 <score>=0.77640465723
#metrics {"Metrics": {"update.time": {"count": 1, "max": 581.0799598693848, "sum": 581.0799598693848, "min": 581.0799598693848}}, "EndTime": 1584056678.83359, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056678.251797}

[03/12/2020 23:44:38 INFO 140601087346496] #progress_metric: host=algo-1, completed 63 % of epochs
#metrics {"Metrics":

[2020-03-12 23:44:48.807] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 160, "duration": 595, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:48 INFO 140601087346496] #quality_metric: host=algo-1, epoch=79, train binary_classification_accuracy <score>=0.747164835165
[03/12/2020 23:44:48 INFO 140601087346496] #quality_metric: host=algo-1, epoch=79, train binary_classification_cross_entropy <loss>=0.521340093927
[03/12/2020 23:44:48 INFO 140601087346496] #quality_metric: host=algo-1, epoch=79, train binary_f_1.000 <score>=0.777369225708
#metrics {"Metrics": {"update.time": {"count": 1, "max": 597.1648693084717, "sum": 597.1648693084717, "min": 597.1648693084717}}, "EndTime": 1584056688.80818, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056688.210233}

[03/12/2020 23:44:48 INFO 140601087346496] #progress_metric: host=algo-1, completed 80 % of epochs
#metrics {"Metrics"


2020-03-12 23:45:10 Uploading - Uploading generated training model
2020-03-12 23:45:10 Completed - Training job completed
[2020-03-12 23:44:58.857] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 194, "duration": 584, "num_examples": 91, "num_bytes": 5796480}
[03/12/2020 23:44:58 INFO 140601087346496] #quality_metric: host=algo-1, epoch=96, train binary_classification_accuracy <score>=0.749252747253
[03/12/2020 23:44:58 INFO 140601087346496] #quality_metric: host=algo-1, epoch=96, train binary_classification_cross_entropy <loss>=0.515487213135
[03/12/2020 23:44:58 INFO 140601087346496] #quality_metric: host=algo-1, epoch=96, train binary_f_1.000 <score>=0.779045221265
#metrics {"Metrics": {"update.time": {"count": 1, "max": 586.4729881286621, "sum": 586.4729881286621, "min": 586.4729881286621}}, "EndTime": 1584056698.857648, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1584056698.270

Training seconds: 112
Billable seconds: 112
CPU times: user 655 ms, sys: 44.9 ms, total: 700 ms
Wall time: 4min 12s


### 배포
배포 또한, 매우 간단하게 `deploy()` 메소드로 수행하실 수 있습니다. 배포는 약 5분에서 10분이 소요됩니다.

In [13]:
%%time
instance_type_inference = 'ml.m5.large'
fm_predictor = fm.deploy(instance_type=instance_type_inference, initial_instance_count=1)

-----------!CPU times: user 204 ms, sys: 8.26 ms, total: 213 ms
Wall time: 5min 31s


In [14]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [15]:
result = fm_predictor.predict(X_test[1000:1010].toarray())
print(result)
print (Y_test[1000:1010])

{'predictions': [{'score': 0.6734210848808289, 'predicted_label': 1.0}, {'score': 0.19272500276565552, 'predicted_label': 0.0}, {'score': 0.23256689310073853, 'predicted_label': 0.0}, {'score': 0.608762264251709, 'predicted_label': 1.0}, {'score': 0.5399172306060791, 'predicted_label': 1.0}, {'score': 0.15502098202705383, 'predicted_label': 0.0}, {'score': 0.38825932145118713, 'predicted_label': 0.0}, {'score': 0.5133833289146423, 'predicted_label': 1.0}, {'score': 0.3564694821834564, 'predicted_label': 0.0}, {'score': 0.11903449147939682, 'predicted_label': 0.0}]}
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]



#### 지금까지 기본적인 사용법을 알아보았으며, 여기에서 실습을 종료하셔도 됩니다. 실습을 일찍 끝내셨거나, 좀 더 깊은 내용을 원하신다면 아래 셀들을 순차적으로 실행해 주세요. ####


#### [주의] 실시간 예측을 제공하기 위해 엔드포인트를 계속 실행할 필요가 없는 경우, 과금을 막기 위해 엔드포인트를 삭제해 주세요. #### 
<br>

## 3. (Optional) top-k 추천을 위하여 FM 모델의 모델 파라메터를 사용하여 knn으로 훈련 및 배포하기
---

이제 SageMaker에 모델을 작성하고 저장 했으므로 동일한 FM 모델을 다운로드하여 KNN 모델에 맞게 다시 패키지 할 수 있습니다. 

### 모델 아티팩트 다운로드

In [16]:
#!pip install mxnet # 필요한 경우 주석을 해제하여 mxnet을 설치해 주세요
import mxnet as mx 
model_file_name = "model.tar.gz"
model_full_path = fm.output_path + "/" + fm.latest_training_job.job_name + "/output/" + model_file_name
print("Model Path: ", model_full_path)

# FM 모델 아티팩트(model.tar.gz) 다운로드 
os.system("aws s3 cp " + model_full_path+ " .")

# 모델 아티팩트 압축 해제
os.system("tar xzvf " + model_file_name)
os.system("unzip -o model_algo-1")
os.system("mv symbol.json model-symbol.json")
os.system("mv params model-0000.params")

Model Path:  s3://sagemaker-us-east-1-143656149352/fm-hol/output/factorization-machines-2020-03-12-23-41-11-305/output/model.tar.gz


0

### 모델 데이터 분리

FM에서 훈련한 파라메터 튜플 ($w_{0}, \mathbf{w}, \mathbf{V}$)을 가져옵니다.

In [17]:
# 모델 추출
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy() # 2625 x 64
w = m._arg_params['w1_weight'].asnumpy() # 2625 x1
b = m._arg_params['w0_weight'].asnumpy() # 1
print(V.shape, w.shape, b.shape)

(2625, 64) (2625, 1) (1,)


### 데이터셋 재가공

이제 FM 모델에서 추출한 모델 파라메터를 다시 패키지하여 k-NN 모델을 훈련하기 위한 준비를 수행해 보겠습니다. W
이 프로세스는 두 개의 데이터셋을 생성합니다.

- Item latent 행렬: k-NN 모델 학습에 사용; $a_i = concat(V, \; w)$
- User latent 행렬: 추론에 사용; $a_u = concat(V, \; 1)$

참고로, 본 핸즈온 코드는 user 및 item ID가 있는 시나리오에만 적용됩니다. 그러나 실제 데이터에는 추가 메타데이터(예: user의 경우 나이, 우편번호, 성별이 포함되고 영화의 경우 영화 장르, 주요 키워드)들이 포함될 수 있습니다. 이러한 경우에는 아래 방법으로 user 및 item 벡터를 추출할 수 있습니다.

- item과 item feature를 $x_i$로 인코딩 후 $\mathbf{V}, \mathbf{w}$에 내적; $a_i = concat(V^T \cdot x_i , \; w^T \cdot x_i)$
- user와 user feature를 $x_u$로 인코딩 후 $\mathbf{V}$에 내적; $a_u = concat(V^T \cdot x_u, \; , 1)$

$a_i$를 사용하여 k-NN 모델을 훈련하고 $a_u$를 사용하여 추론을 수행하시면 됩니다.

In [18]:
# item latent matrix - concat(V[i], w[i]).  
knn_item_matrix = np.concatenate((V[nb_users:], w[nb_users:]), axis=1) # 1682 x 65
knn_train_label = np.arange(1,nb_movies+1) # [1, 2, 3, ..., 1681, 1682]

# user latent matrix - concat (V[u], 1) 
ones = np.ones(nb_users).reshape((nb_users, 1)) # 943x1
knn_user_matrix = np.concatenate((V[:nb_users], ones), axis=1) # 943 x 65

### k-NN 모델 훈련

k-NN 모델은 기본 index_type (faiss.Flat)을 사용합니다.  대규모 데이터셋의 경우 속도가 느려지기에, 이런 경우 더 빠른 훈련을 위해 다른 index_type 매개 변수를 사용할 수 있습니다. index 유형에 대한 자세한 내용은 k-NN 설명서를 참조해 주세요.

In [19]:
print('KNN train features shape = ', knn_item_matrix.shape)
knn_prefix = 'knn'
knn_output_prefix  = 's3://{}/{}/output'.format(bucket, knn_prefix)
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket, knn_prefix, train_key, "dense", knn_train_label)
print('Uploaded KNN train data: {}'.format(knn_train_data_path))

KNN train features shape =  (1682, 65)
Uploaded KNN train data: s3://sagemaker-us-east-1-143656149352/knn/train.protobuf


In [20]:
nb_recommendations = 100

knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type=instance_type_training,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session())

knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1], k=nb_recommendations, 
                        index_metric="INNER_PRODUCT", predictor_type='classifier', sample_size=200000)
fit_input = {'train': knn_train_data_path}

훈련을 시작합니다. 아래 셀의 수행 시간은 약 4분에서 5분이 소요됩니다.

In [21]:
%%time
knn.fit(fit_input)
knn_model_name =  knn.latest_training_job.job_name
print("Created model: ", knn_model_name)

2020-03-12 23:57:36 Starting - Starting the training job...
2020-03-12 23:57:38 Starting - Launching requested ML instances......
2020-03-12 23:59:06 Starting - Preparing the instances for training.........
2020-03-13 00:00:13 Downloading - Downloading input data...
2020-03-13 00:01:06 Training - Training image download completed. Training in progress.
2020-03-13 00:01:06 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
[03/13/2020 00:00:59 INFO 139648235816768] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'index_metric': u'L2', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'faiss_index_ivf_nlists': u'auto', u'epochs': u'1', u'index_type': u'faiss.Flat', u'_faiss_index_nprobe': u'5', u'_kvstore': u'dist_async', u'_num_kv_servers': u'1', u'mini_batch_size': u'5000'}
[03/13/2020 00:00:59 INFO 139648235816768] Reading provided configurati


2020-03-13 00:01:12 Completed - Training job completed
Training seconds: 59
Billable seconds: 59
Created model:  knn-2020-03-12-23-57-36-380
CPU times: user 513 ms, sys: 21.1 ms, total: 534 ms
Wall time: 4min 12s


배치 추론에서 참조할 수 있도록 모델을 생성합니다.

In [22]:
# 다음 단계에서 배치 추론 중에 참조할 수 있도록 모델 저장
sm = boto3.client(service_name='sagemaker')
primary_container = {
    'Image': knn.image_name,
    'ModelDataUrl': knn.model_data,
}

knn_model = sm.create_model(
        ModelName = knn.latest_training_job.job_name,
        ExecutionRoleArn = knn.role,
        PrimaryContainer = primary_container)

### Batch Transform

Amazon SageMaker의 Batch Transform 기능을 사용하면 대규모로 배치 추론 결과를 생성할 수 있습니다. <br>
아래 셀의 실행이 완료되기까지는 약 4분 소요됩니다.

In [23]:
%%time
# 추론 데이터 S3에 업로드
knn_batch_data_path = writeDatasetToProtobuf(knn_user_matrix, bucket, knn_prefix, train_key, "dense")
print("Batch inference data path: ", knn_batch_data_path)

# Transformer 객체 초기화
transformer = sagemaker.transformer.Transformer(
    base_transform_job_name="knn",
    model_name=knn_model_name,
    instance_count=1,
    instance_type=instance_type_inference,
    output_path=knn_output_prefix,
    accept="application/jsonlines; verbose=true"
)

# 변환 작업 시작
transformer.transform(knn_batch_data_path, content_type='application/x-recordio-protobuf')
transformer.wait()

# S3에서 출력 파일 다운로드
results_file_name = "inference_output"
inference_output_file = "knn/output/train.protobuf.out"
s3_client = boto3.client('s3')
s3_client.download_file(bucket, inference_output_file, results_file_name)
with open(results_file_name) as f:
    results = f.readlines()  

Batch inference data path:  s3://sagemaker-us-east-1-143656149352/knn/train.protobuf
..................Docker entrypoint called with argument(s): serve
[03/13/2020 00:04:37 INFO 140135089923904] loaded entry point class algorithm.serve.server_config:config_api
[03/13/2020 00:04:37 INFO 140135089923904] loading entry points
[03/13/2020 00:04:37 INFO 140135089923904] loaded request iterator text/csv
[03/13/2020 00:04:37 INFO 140135089923904] loaded request iterator application/x-recordio-protobuf
[03/13/2020 00:04:37 INFO 140135089923904] loaded request iterator application/json
[03/13/2020 00:04:37 INFO 140135089923904] loaded request iterator application/jsonlines
[03/13/2020 00:04:37 INFO 140135089923904] loaded response encoder application/x-recordio-protobuf
[03/13/2020 00:04:37 INFO 140135089923904] loaded response encoder application/json
[03/13/2020 00:04:37 INFO 140135089923904] loaded response encoder application/jsonlines
[03/13/2020 00:04:38 INFO 140135089923904] loaded entry

### top-k 추론 예시
배치 추론 결과에서 90번 사용자의 추천 영화를 확인해 보겠습니다. 결과 데이터프레임의 1번째 행은 영화 id, 2번째 행은 영화 제목, 3번째 행은
유사도입니다. 

In [24]:
def get_movie_title(movie_id):
    movie_id = int(movie_id)
    return items.iloc[movie_id]['TITLE']

import json
test_user_idx = 89 # 인덱스는 0부터 시작하므로 90번 사용자의 인덱스는 89입니다.
u_one_json = json.loads(results[test_user_idx])

items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

movie_id_list = [int(movie_id) for movie_id in u_one_json['labels']]
movie_dist_list = [round(distance, 4) for distance in u_one_json['distances']]
movie_title_list = [get_movie_title(movie_id) for movie_id in movie_id_list]

recommend_df = pd.DataFrame({'movie_id': movie_id_list, 
                             'movie_title': movie_title_list, 
                             'movie_dist': movie_dist_list})
print("Recommendations for user: ", test_user_idx)
recommend_df.head(30)

Recommendations for user:  89


,movie_id,movie_title,movie_dist
0,423,Children of the Corn: The Gathering (1996),2.5400
1,165,Manon of the Spring (Manon des sources) (1986),2.5569
2,505,Rebel Without a Cause (1955),2.5701
3,87,Sleepless in Seattle (1993),2.5762
4,966,Little Lord Fauntleroy (1936),2.5800
5,429,Duck Soup (1933),2.5837
6,655,M (1931),2.5857
7,89,So I Married an Axe Murderer (1993),2.5864
8,69,Four Weddings and a Funeral (1994),2.5917
9,23,Rumble in the Bronx (1995),2.5937
